In [0]:
class Bronze():
    def __init__(self):
        self.base_data_dir = "/FileStore/data_spark_streaming_scholarnest"
        self.BOOTSTRAP_SERVER = "pkc-9q8rv.ap-south-2.aws.confluent.cloud:9092"
        self.JAAS_MODULE = "org.apache.kafka.common.security.plain.PlainLoginModule"
        self.CLUSTER_API_KEY = "GP7UYZSUZXLFHNRO"
        self.CLUSTER_API_SECRET = "hS3z7pXZ2e+znqqh2Jf/RNsVShHYuYCFW3DKkG81FfB4Rwt/TL39ujBH9dporeGI"

    def getSchema(self):
        return """InvoiceNumber string, CreatedTime bigint, StoreID string, PosID string, CashierID string,
                CustomerType string, CustomerCardNo string, TotalAmount double, NumberOfItems bigint, 
                PaymentMethod string, TaxableAmount double, CGST double, SGST double, CESS double, 
                DeliveryType string,
                DeliveryAddress struct<AddressLine string, City string, ContactNumber string, PinCode string, 
                State string>,
                InvoiceLineItems array<struct<ItemCode string, ItemDescription string, 
                    ItemPrice double, ItemQty bigint, TotalValue double>>
            """

    def ingestFromKafka(self, startingTime = 1):
        return ( spark.readStream
                        .format("kafka")            
                        .option("kafka.bootstrap.servers", self.BOOTSTRAP_SERVER)
                        .option("kafka.security.protocol", "SASL_SSL")
                        .option("kafka.sasl.mechanism", "PLAIN")
                        .option("kafka.sasl.jaas.config", "kafkashaded.org.apache.kafka.common.security.plain.PlainLoginModule required username='{}' password='{}';".format(self.CLUSTER_API_KEY, self.CLUSTER_API_SECRET))
                        # .option("kafka.sasl.jaas.config", f"{self.JAAS_MODULE} required username='{self.CLUSTER_API_KEY}' password='{self.CLUSTER_API_SECRET}';")
                        .option("subscribe", "Danaerys")
                        .option("kafka.ssl.endpoint.identification.algorithm", "https")
                        # .option("maxOffsetsPerTrigger", 10)
                        .option("startingTimestamp", startingTime)
                        .load()
                )
       
    
    def getInvoices(self, kafka_df):
        from pyspark.sql.functions import cast, from_json
        return (kafka_df.select(kafka_df.key.cast("string").alias("key"),
                            from_json(kafka_df.value.cast("string"), self.getSchema()).alias("value"),
                            "topic", "timestamp")
                )

    def process(self, startingTime = 1):
        print(f"Starting Bronze Stream...", end='')
        rawDF = self.ingestFromKafka(startingTime)
        invoicesDF = self.getInvoices(rawDF)
        sQuery =  ( invoicesDF.writeStream
                            .queryName("bronze-ingestion")
                            .option("checkpointLocation", f"{self.base_data_dir}/chekpoint/invoices_bz")
                            .outputMode("append")
                            .toTable("invoices_bz")           
                    ) 
        print("Done")
        return sQuery   

In [0]:
model = Bronze()
kafka_df = model.ingestFromKafka()
modified_df = model.getInvoices(kafka_df)
model.process()

Starting Bronze Stream...Done
Out[14]: <pyspark.sql.streaming.query.StreamingQuery at 0x7fb92ab2eb50>

In [0]:
df = spark.sql("select * from invoices_bz")

In [0]:
print(df.count())

1458644
